In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import seaborn as sns
import re

In [3]:
#Read in the original csv file
KansasCity311 = pd.read_csv('../data/Kansas_City_MO_311.csv', low_memory=False)

In [4]:
KansasCity311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960502 entries, 0 to 960501
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   CASE ID                 960502 non-null  int64  
 1   SOURCE                  960488 non-null  object 
 2   DEPARTMENT              960502 non-null  object 
 3   WORK GROUP              960502 non-null  object 
 4   REQUEST TYPE            960502 non-null  object 
 5   CATEGORY                960502 non-null  object 
 6   TYPE                    960502 non-null  object 
 7   DETAIL                  960502 non-null  object 
 8   CREATION DATE           960502 non-null  object 
 9   CREATION MONTH          960502 non-null  int64  
 10  CREATION YEAR           960502 non-null  int64  
 11  STATUS                  960502 non-null  object 
 12  EXCEEDED EST TIMEFRAME  960487 non-null  object 
 13  CLOSED DATE             929529 non-null  object 
 14  CLOSED MONTH        

In [5]:
#convert times to datetimes
KansasCity311['CREATION DATE'] = pd.to_datetime(KansasCity311['CREATION DATE'])

In [8]:
#what is the date range covered in this current dataframe?
#2006 - June 2016
KansasCity311 = KansasCity311.sort_values(['CREATION DATE'],ascending=True)
KansasCity311.tail()

,CASE ID,SOURCE,DEPARTMENT,WORK GROUP,REQUEST TYPE,CATEGORY,TYPE,DETAIL,CREATION DATE,CREATION MONTH,...,DAYS TO CLOSE,STREET ADDRESS,ADDRESS WITH GEOCODE,ZIP CODE,NEIGHBORHOOD,COUNCIL DISTRICT,PARCEL ID NO,XCOORDINATE,YCOORDINATE,CASE URL
960348,2016073840,EMAIL,Health,Health-Noise Control-,Public Health-Noise-All,Public Health,Noise,All,2016-06-28,6,...,0.0,8406 NE 89TH PL,"8406 NE 89TH PL\n64157\n(39.254649, -94.478009)",64157.0,Shoal Creek,1.0,226022,2.794935e+06,1.124485e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
960299,2016073759,PHONE,Parks and Rec,Parks and Rec-Landscape Services-Forestry,Trees-Storm Damage-Trim,Trees,Storm Damage,Trim,2016-06-28,6,...,NaN,7242 FLORA AVE,"7242 FLORA AVE\n64131\n(38.995118, -94.569747)",64131.0,East Meyer 6,5.0,114506,2.768820e+06,1.029938e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
960339,2016073828,PHONE,NHS,NHS-Solid Waste-Administration,Trash / Recycling-Services-Service Issue / Pro...,Trash / Recycling,Services,Service Issue / Problem,2016-06-28,6,...,0.0,2127 PARK AVE,"2127 PARK AVE\n64127\n(39.085799, -94.555535)",64127.0,Wendell Phillips,3.0,13080,2.773045e+06,1.062919e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
960351,2016073844,PHONE,NHS,NHS-Animal Health and Safety-,Animals / Pets-Cruelty or Neglect-Inclement We...,Animals / Pets,Cruelty or Neglect,Inclement Weather,2016-06-28,6,...,NaN,622 NE 41ST TER,"622 NE 41ST TER\n64116\n(39.169171, -94.573066)",64116.0,River View,4.0,84497,2.767988e+06,1.093398e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
960501,2016074055,PHONE,Public Works,Public Works-Street and Traffic-District 2,Streets / Roadways / Alleys-Street Clean Up-Di...,Streets / Roadways / Alleys,Street Clean Up,District 2,2016-06-28,6,...,NaN,5516 E 35TH ST,"5516 E 35TH ST\n64128\n(39.061527, -94.519244)",64128.0,Dunbar,3.0,23495,2.783258e+06,1.054167e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...


In [9]:
#Cicadas came to Missouri in 2016, via two broods at once: Brood XXIII and Brood IV.  
#Kansas City is in Jackson County which has confirmed sightings

KansasCity311trim = KansasCity311[KansasCity311['CREATION DATE'] > dt.datetime(2013,5,1)]
KansasCity311trim = KansasCity311trim[KansasCity311trim['CREATION DATE'] < dt.datetime(2017,7,1)]

## Isolating data to months of May and June for each year.  Cicada time.



In [14]:
#2013
KC2013 = KansasCity311trim[KansasCity311trim['CREATION DATE'] > dt.datetime(2013,5,1)]
KC2013 = KC2013[KC2013['CREATION DATE'] < dt.datetime(2013,7,1)]
KC2013 = KC2013.sort_values(['CREATION DATE'],ascending=False)
KC2013

,CASE ID,SOURCE,DEPARTMENT,WORK GROUP,REQUEST TYPE,CATEGORY,TYPE,DETAIL,CREATION DATE,CREATION MONTH,...,DAYS TO CLOSE,STREET ADDRESS,ADDRESS WITH GEOCODE,ZIP CODE,NEIGHBORHOOD,COUNCIL DISTRICT,PARCEL ID NO,XCOORDINATE,YCOORDINATE,CASE URL
684119,2013091072,WEB,Public Works,Public Works-Street and Traffic-Rural Mowing,Rural ROW Mowing,Weeds,Right of Way,All,2013-06-30,6,...,11.0,9523 N KENWOOD AVE,"9523 N KENWOOD AVE\n64155\n(39.267292, -94.571...",64155.0,Gashland,2.0,99970,2.768570e+06,1.129011e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684129,2013091087,WEB,NHS,NHS-Neighborhood Preservation-,Property Violations,Property Violations,Private Property,All,2013-06-30,6,...,1.0,4545 CHESTNUT AVE,"4545 CHESTNUT AVE\n64130\n(39.042927, -94.551962)",64130.0,Oak Park Southwest,3.0,34852,2.774033e+06,1.047269e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684138,2013091096,WEB,Public Works,Public Works-Solid Waste-,Dead Animal Pick-up,Trash,Animal,Dead,2013-06-30,6,...,2.0,100 W 90TH TER,"100 W 90TH TER\n64114\n(38.964107, -94.592324)",64114.0,Boone Hills,6.0,150094,2.762502e+06,1.018692e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684108,2013091054,WEB,NHS,NHS-Neighborhood Preservation-,Property Violations,Property Violations,Private Property,All,2013-06-30,6,...,79.0,1201 E 76TH ST,"1201 E 76TH ST\n64131\n(38.989414, -94.574405)",64131.0,East Meyer 6,5.0,116089,2.767552e+06,1.027705e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684123,2013091077,PHONE,Parks and Rec,Parks and Rec-Landscape Services-Forestry,Tree Removal of a City Tree,Trees,Removal,*Select one,2013-06-30,6,...,1.0,4800 E 63RD ST,4800 E 63RD ST\n64130\n,64130.0,Swope Parkway-Elmwood,5.0,492,2.780203e+06,1.036936e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665581,2013057972,PHONE,Public Works,Public Works-Solid Waste-,Trash Recycle Missed by Contractor North,Trash,Recycle,North Contractor,2013-05-02,5,...,4.0,5113 NW COVES DR,"5113 NW COVES DR\n64151\n(39.245699, -94.639684)",64151.0,The Coves,2.0,161281,2.749067e+06,1.121152e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
665582,2013057974,PHONE,NHS,NHS-Animal Health and Safety-,Stray Animal Confined,Animal,Stray Confined,Confined,2013-05-02,5,...,1.0,8710 N WHEATON CT,"8710 N WHEATON CT\n64153\n(39.253042, -94.698094)",64153.0,KCI & 2nd Creek,2.0,167696,2.732532e+06,1.123872e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
665583,2013057975,PHONE,NHS,NHS-Animal Health and Safety-,Animal At Large,Animal,Owned or Stray at Large,All,2013-05-02,5,...,12.0,2000 BALTIMORE AVE,"2000 BALTIMORE AVE\n64108\n(39.088902, -94.584...",64108.0,Crossroads,4.0,124949,2.764533e+06,1.064071e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
665576,2013057951,PHONE,NHS,NHS-Neighborhood Preservation-,Property Violations,Property Violations,Private Property,All,2013-05-02,5,...,5.0,4000 OLIVE ST,"4000 OLIVE ST\n64130\n(39.053572, -94.555989)",64130.0,Ivanhoe Southeast,3.0,136441,2.772705e+06,1.051183e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...


In [15]:
#2014
KC2014 = KansasCity311trim[KansasCity311trim['CREATION DATE'] > dt.datetime(2014,5,1)]
KC2014 = KC2014[KC2014['CREATION DATE'] < dt.datetime(2014,7,1)]

In [16]:
#2015 Cicada year
KC2015 = KansasCity311trim[KansasCity311trim['CREATION DATE'] > dt.datetime(2015,5,1)]
KC2015 = KC2015[KC2015['CREATION DATE'] < dt.datetime(2015,7,1)]

In [17]:
#2016
KC2016 = KansasCity311trim[KansasCity311trim['CREATION DATE'] > dt.datetime(2016,5,1)]
KC2016 = KC2016[KC2016['CREATION DATE'] < dt.datetime(2016,7,1)]

In [27]:
KC2016.head()

,CASE ID,SOURCE,DEPARTMENT,WORK GROUP,REQUEST TYPE,CATEGORY,TYPE,DETAIL,CREATION DATE,CREATION MONTH,...,DAYS TO CLOSE,STREET ADDRESS,ADDRESS WITH GEOCODE,ZIP CODE,NEIGHBORHOOD,COUNCIL DISTRICT,PARCEL ID NO,XCOORDINATE,YCOORDINATE,CASE URL
940449,2016046200,WEB,KCPD,KCPD-Parking Control-,Vehicles / Parking-Abandoned On Street-Car,Vehicles / Parking,Abandoned On Street,Car,2016-05-02,5,...,5.0,7800 N STRATHBURY AVE,"7800 N STRATHBURY AVE\n64151\n(39.236261, -94....",64151.0,Platte Brook North,2.0,162277,2.751708e+06,1.117738e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
940450,2016046188,PHONE,NHS,NHS-Animal Health and Safety-,Animals / Pets-Stray-Confined,Animals / Pets,Stray,Confined,2016-05-02,5,...,16.0,6917 NW 80TH ST,"6917 NW 80TH ST\n64152\n(39.239672, -94.66035)",64152.0,Platte Ridge,2.0,169333,2.743297e+06,1.118872e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
940451,2016046214,PHONE,NHS,NHS-Solid Waste-,Trash / Recycling-Trash Collection-Missed by C...,Trash / Recycling,Trash Collection,Missed by Contractor South,2016-05-02,5,...,1.0,9429 WEST LONGVIEW PKWY,"9429 WEST LONGVIEW PKWY\n64138\n(38.952722, -9...",64138.0,White Oak,5.0,68183,2.793604e+06,1.014440e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
155679,2016045750,PHONE,NHS,NHS-Neighborhood Preservation-,Property Violations,Mowing / Weeds,Private / Commercial,Lot - Structure,2016-05-02,5,...,NaN,1609 NW 67TH ST,"1609 NW 67TH ST\n64118\n(39.215302, -94.599425)",64118.0,Clayton,2.0,96240,2.760535e+06,1.109986e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
940039,2016045862,PHONE,NHS,NHS-Solid Waste-,Animals / Pets-Dead Animal -Street,Animals / Pets,Dead Animal,Street,2016-05-02,5,...,1.0,3341 COLLEGE AVE,"3341 COLLEGE AVE\n64128\n(39.064603, -94.545035)",64128.0,Oak Park Northwest,3.0,25554,2.776012e+06,1.055186e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...


In [25]:
#2017 - NO DATA!!!!!!
KC2017 = KansasCity311trim[KansasCity311trim['CREATION DATE'] > dt.datetime(2017,5,1)]
KC2017 = KC2017[KC2017['CREATION DATE'] < dt.datetime(2017,7,1)]

In [26]:
KC2017.head

<bound method NDFrame.head of Empty DataFrame
Columns: [CASE ID, SOURCE, DEPARTMENT, WORK GROUP, REQUEST TYPE, CATEGORY, TYPE, DETAIL, CREATION DATE, CREATION MONTH, CREATION YEAR, STATUS, EXCEEDED EST TIMEFRAME, CLOSED DATE, CLOSED MONTH, CLOSED YEAR, DAYS TO CLOSE, STREET ADDRESS, ADDRESS WITH GEOCODE, ZIP CODE, NEIGHBORHOOD, COUNCIL DISTRICT, PARCEL ID NO, XCOORDINATE, YCOORDINATE, CASE URL]
Index: []

[0 rows x 26 columns]>

## Bring it into a single dataframe

In [28]:
years = [KC2013, KC2014, KC2015, KC2016]
KC311_final = pd.concat(years)

In [23]:
KC311_final = KC311_final.sort_values(['CREATION DATE'],ascending=True)

In [29]:
KC311_final

,CASE ID,SOURCE,DEPARTMENT,WORK GROUP,REQUEST TYPE,CATEGORY,TYPE,DETAIL,CREATION DATE,CREATION MONTH,...,DAYS TO CLOSE,STREET ADDRESS,ADDRESS WITH GEOCODE,ZIP CODE,NEIGHBORHOOD,COUNCIL DISTRICT,PARCEL ID NO,XCOORDINATE,YCOORDINATE,CASE URL
684119,2013091072,WEB,Public Works,Public Works-Street and Traffic-Rural Mowing,Rural ROW Mowing,Weeds,Right of Way,All,2013-06-30,6,...,11.0,9523 N KENWOOD AVE,"9523 N KENWOOD AVE\n64155\n(39.267292, -94.571...",64155.0,Gashland,2.0,99970,2.768570e+06,1.129011e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684129,2013091087,WEB,NHS,NHS-Neighborhood Preservation-,Property Violations,Property Violations,Private Property,All,2013-06-30,6,...,1.0,4545 CHESTNUT AVE,"4545 CHESTNUT AVE\n64130\n(39.042927, -94.551962)",64130.0,Oak Park Southwest,3.0,34852,2.774033e+06,1.047269e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684138,2013091096,WEB,Public Works,Public Works-Solid Waste-,Dead Animal Pick-up,Trash,Animal,Dead,2013-06-30,6,...,2.0,100 W 90TH TER,"100 W 90TH TER\n64114\n(38.964107, -94.592324)",64114.0,Boone Hills,6.0,150094,2.762502e+06,1.018692e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684108,2013091054,WEB,NHS,NHS-Neighborhood Preservation-,Property Violations,Property Violations,Private Property,All,2013-06-30,6,...,79.0,1201 E 76TH ST,"1201 E 76TH ST\n64131\n(38.989414, -94.574405)",64131.0,East Meyer 6,5.0,116089,2.767552e+06,1.027705e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684123,2013091077,PHONE,Parks and Rec,Parks and Rec-Landscape Services-Forestry,Tree Removal of a City Tree,Trees,Removal,*Select one,2013-06-30,6,...,1.0,4800 E 63RD ST,4800 E 63RD ST\n64130\n,64130.0,Swope Parkway-Elmwood,5.0,492,2.780203e+06,1.036936e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960348,2016073840,EMAIL,Health,Health-Noise Control-,Public Health-Noise-All,Public Health,Noise,All,2016-06-28,6,...,0.0,8406 NE 89TH PL,"8406 NE 89TH PL\n64157\n(39.254649, -94.478009)",64157.0,Shoal Creek,1.0,226022,2.794935e+06,1.124485e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
960299,2016073759,PHONE,Parks and Rec,Parks and Rec-Landscape Services-Forestry,Trees-Storm Damage-Trim,Trees,Storm Damage,Trim,2016-06-28,6,...,NaN,7242 FLORA AVE,"7242 FLORA AVE\n64131\n(38.995118, -94.569747)",64131.0,East Meyer 6,5.0,114506,2.768820e+06,1.029938e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
960339,2016073828,PHONE,NHS,NHS-Solid Waste-Administration,Trash / Recycling-Services-Service Issue / Pro...,Trash / Recycling,Services,Service Issue / Problem,2016-06-28,6,...,0.0,2127 PARK AVE,"2127 PARK AVE\n64127\n(39.085799, -94.555535)",64127.0,Wendell Phillips,3.0,13080,2.773045e+06,1.062919e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
960351,2016073844,PHONE,NHS,NHS-Animal Health and Safety-,Animals / Pets-Cruelty or Neglect-Inclement We...,Animals / Pets,Cruelty or Neglect,Inclement Weather,2016-06-28,6,...,NaN,622 NE 41ST TER,"622 NE 41ST TER\n64116\n(39.169171, -94.573066)",64116.0,River View,4.0,84497,2.767988e+06,1.093398e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...


## Get the relevant request types

In [31]:
KC311_final['REQUEST TYPE'].unique()

array(['Rural ROW Mowing', 'Property Violations', 'Dead Animal Pick-up',
       'Tree Removal of a City Tree', 'Barking Dog',
       'Traffic Sign Down or Missing - Emergency',
       'SLG3 Streetlights Out - Two to Four Lights', 'Noise Complaints',
       'Abandoned Vehicle', 'Animal Investigation',
       'Park Maintenance North Region',
       'Sign Missing or Damaged - Non-emergency',
       'Trash Missed by Contractor North',
       'Water Leak or Pressure Problem',
       'Street Maintenance General (South of 47th Street and West of Blue PKWY)',
       'Wildlife', 'SLG2 Streetlight Burning During Day',
       'Traffic Signal Out', 'Food Establishment Complaints',
       'Graffiti on Private Property', 'Animal Control Administration',
       'Sewer Pavement Restoration Referral',
       'Pothole (River south to 47th Street and East of Blue PKWY)',
       'Bridge Repair (All Districts)', 'Traffic Signal Malfunction',
       'Sewer Water in Basement - Referral',
       'Vacant Struc

In [ ]:
# What types of Phone Calls indicate bad behavior and/or an annoyed populace?  Excluding Emergency-related things and burnt-out streetlights
relevant_requests = [
    'Rural ROW Mowing', 'SLD0 Streetlight Out - Multiple Outage of Five or More','Property Violations', 'Building Code and Permit Violations',
    'Dead Animal Pick-up', 'Street Maintenance General (North of the River)','Public Health-Noise-Other',
     'Tree Removal of a City Tree', 'Pothole (North of River)', 'Ditch Cleaning (North from 27th Street )', 'Sidewalks / Curbs / Ditch-Damage / Dis-Repair-City',
     'Barking Dog', 'Parking Ticket Complaint', 'Animals / Pets-Bite-Owned', 'Animals / Pets-Cruelty or Neglect-Shelter / Food / Water',
      'Noise Complaints', 'Mowing on the Right of Way', 'Alley Cleaning (South of 47th Street and West of BLue PKWY)', 'Signs-Traffic Sign-Missing � Non-Emergency'
     'Abandoned Vehicle', 'Sign Violation Zoning', 'Graffiti on City Property', 'Mud and silt', 'Mowing / Weeds-Parks-North',
       'SLA0 Streetlight Pole Down or Leaning', 'Animal Bite', 'Public Restroom Sanitation','Trash-Staff Compliments for Solid Waste', 'Trash / Recycling-Nuisance-Early Set Out',
      'Animal Investigation','Sign Missing or Damaged - Non-emergency','Trash Missed by Contractor North',
    'Water Leak or Pressure Problem','Wildlife', 'SLG2 Streetlight Burning During Day','Animals / Pets-Questionable Activity-Cock fighting',
       'Animals / Pets-Stray-At Large - Highway', 'Public Health-Noise-Music', 'Public Health-Noise-Vehicles',
       'Trash / Recycling-Dumping-Park - South','Lights / Signals-Traffic Signal-Signal timing','Water-Quality-Taste','Public Health-Smoking / Tobacco-Restaurant',
       'Signs-Street Name Signs-Graffiti','Prop/Build/Construct - Land Use / Zoning - Unapproved Land Use',
       'Animals / Pets-Stray-At Large - Non-Aggressive', 'Trash / Recycling-Trash Collection-Missed by Contractor North','Trash / Recycling-Leaf / Brush-Missed Pick Up',
       'Streets / Roadways / Alleys-Broken Asphalt-District 2', 'Vehicles / Parking-Abandoned On Street-Car', 'Animals / Pets-Cruelty or Neglect-Abuse / Injury / Death',
       'Animals / Pets-Cruelty or Neglect-Inclement Weather', 'Trash / Recycling-Recycling-Missed by Contractor North','Vehicles / Parking-Abandoned On Street-Boat',
       'Trash / Recycling-Dumping-Camera Request', 'Lights / Signals-Street Light-Burning during the day','Public Health-Hotel / Motel-Unsanitary Conditions',
       'Animals / Pets-Services-Barking Dog Letter', 'Animals / Pets-Dead Animal -Curb', 'Mowing / Weeds-Alley-District 3','Vehicles / Parking-Abandoned On Street-Trailer',
       'Trash / Recycling-Bulky Pick Up -Missed Pick Up','Animals / Pets-Cruelty or Neglect-Abandonment','Trash / Recycling-Recycling-Repeat Missed',
       'Trash / Recycling-Recycling-Missed by City', 'Animals / Pets-Services-Off Leash - Investigation', 'Parks & Recreation-Graffiti-North',
       'Trash / Recycling-Services-Feedback', 'Signs-Street Name Signs-Missing','Mowing / Weeds - Visibility - Intersection',
       'Lights / Signals-Street Light-Graffiti','Public Health-Smoking / Tobacco-Public Building',
       'Mowing / Weeds-Blvd / Pkwy-North','Vehicles / Parking-Illegally Parked-Street/Sidewalk','Public Health-Noise-Trash Truck',
       'Animals / Pets-Questionable Activity-Other', 'Signs-Traffic Sign-Graffiti', 'Public Health-Noise-All'
       'Animals / Pets-Stray-At Large - Owned', 'Trash / Recycling-Recycling-Missed by Contractor South',
       'Animals / Pets-Wildlife-Inside', 'Trash / Recycling-Trash Collection-Repeat Missed', 'Sidewalks / Curbs / Ditch-Unapproved Objects-Other',
       'Trash / Recycling-Dumping-Park - North', 'Vehicles / Parking-Abandoned On Street-Other',
       'Sidewalks / Curbs / Ditch-Unapproved Objects-Basketball Hoop/Goal',
       'Trash / Recycling-Services-Service Issue / Problem',
       'Animals / Pets-Stray-At Large - Aggressive',
      'Food Establishment Complaints', 'Smoking Complaints','Graffiti on Signs', 'Issues on City Maintained Property', 'Streets / Roadways / Alleys-Street Services-Service Issue / Problem',
                     'Graffiti on Private Property', 'Outdoor Air Quality', 'Graffiti on Signs',
                     'Animal Control Administration', 'Tree Pruning for Block', 'Homeless Camp', 'Zoning Issue-Business in a residence',
       'Public Safety-Police-Homeless camp', 'Trash / Recycling-Dumping-Right of Way (ROW)',
                     'Pothole (River south to 47th Street and East of Blue PKWY)', 'Trees-Trimming-Tree Limbs',
                     'SLG0 Streetlight Out - Single Light Out', 'Trash / Recycling-Dumping-Private Property',
                     'SLP0 Streetlight Miscellaneous', 'Prop/Build/Construct-Construction Issue/Concern-Mud / Silt on street',
                     'Tree Trimming for City Tree', 'Traffic Signal Pole Damage, Rust, Vandalism',
                     'Sewer Cleaning or Repair and Waste Water Concern', 'Prop/Build/Construct-Land Use / Zoning Issue-Number of Dwelling Units',
                     'Trash Recycle Missed by Contractor South', 'No Water Bill Received',
       'Trash Cart Damaged','Bulky Pick-up Missed', 'Mowing / Weeds-Public Property-Land Bank', 'Trash / Recycling-Nuisance-Land Bank',
                     'Illegal Dumping on Property', 'Mowing / Weeds-Public Property-City Facility',
                     'Illegal Dumping on Right of Way', 'Mowing / Weeds-Right of Way (ROW)-Region - North',
                     'Trash Missed by Contractor South', 'Trash / Recycling-Dumping-Alley',
                     'Trash - Missed by City', 'Prop/Build/Construct-Construction Issue/Concern-Work without permit',
                     'Alley Cleaning (South of River to 47th Street and East of Blue PKWY)',
                     'Trash Recycle Missed by Contractor North',
                     'Tree Limb or Brush Removal for City Tree', 'Trees-Land Bank-Dead',
                     'Injury or Cruelty Involving an Animal','Animal At Large','Catch Basins and Storm Water Concerns',
       'Bulky Address Validation', 'Trash Recycling Missed by City',
       'Sewer Deodorizing - Referral', 'Violations and Encroachments on the Right of Way', 
                     'Graffiti on Streetlight Pole', 'Dangerous Building', 'Tree Trimming at Intersection', 'Stray Animal Confined',
       'Trash - Staff Complaints for Solid Waste', 'SLG1 Streetlight Blinking Off and On', 'Sidewalk or Curb in Disrepair', 'Bedbugs Problem',
                     'Vehicle Illegally Parked', 'Street Sign Damage', 'Tree Stump Removal for City Tree',
       'Ditch Cleaning (South of 27th Street)', 'Pothole (South of 47th Street and West Of Blue PKWY)', 'Tree Disease, Insects or Vandalism',
    'Vehicles / Parking-Commercial Vehicle-On private property',
       'Government-Public Works-Service Issue / Problem',
       'Trees-Land Bank-Tree Debris', 'Trash / Recycling-Trash Collection-Missed by City', 'Sidewalks / Curbs / Ditch-Unapproved Objects-Landscaping',
       'Mowing / Weeds-Parks-South', 'Trees-Trimming-Seasonal','Mowing / Weeds-Parks-Central','Mowing / Weeds-Blvd / Pkwy-Central',
       'Trees-Services-Service Issue / Problem',
       'Animals / Pets-Stray-Injured','Mowing / Weeds-Right of Way (ROW)-Region - South','Mowing / Weeds-Alley-District 2','Mowing / Weeds-Right of Way (ROW)-Region - Central',
                     
                     
                     
    
]